In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## CMB lensing ##

Lensing kernel vs. z.
Look at linear vs. nonlinear P(k,z).
Look at cumulative C_l^kk.

Other figures from Hanson/Challinor/etc reviews?

### CLASS ###

We'll be using the [Cosmic Linear Anisotropy Solving System](http://class-code.net/) (CLASS) Boltzmann code to compute the $P(k,z)$ which goes into our integrals.

There are two "commonly used" codes which evolve the linearized perturbation equations for cosmology.  These are highly useful tools for working in the CMB or large-scale structure.  The CLASS code is very well documented and maintained, and a public version is avaialble [at this GitHub repo](https://github.com/lesgourg/class_public), making it easy to keep up to date (or to contribute!).

If you clone the repository, e.g. with
`git clone https://github.com/lesgourg/class_public class`
then you can just go into the class directory and "make" the code.  I've never had a problem with this on any machine that a few edits to the Makefile didn't fix.  If you want to install the Python interface just go into the Python directory and use `python setup.py install`.  On the datahub you can install as root so you don't need the `--user`, otherwise you can use your own system.  There is also a `Conda` version if you use an Anaconda python distribution.

In [ ]:
# Import the Class class.
from classy import Class

In [ ]:
# Define the cosmology (what is not specified will be set to CLASS default parameters).
# We'll use the "Planck 2018 cosmology" (plus BAO), defined in the right-hand column
# of Table 6 in https://arxiv.org/pdf/1807.06205.pdf
# We'll ask for matter P(k), using the HaloFit non-linear power spectrum fitting
# function, between z=0 and 10:
params = {
    'output': 'mPk',
    'P_k_max_h/Mpc': 20.,
    'non linear': 'halofit',
    'z_pk': '0.0,10.0',
    'lensing': 'no',
    'A_s': np.exp(3.040)*1e-10,
    'n_s': 0.96824,
    'h': 0.6770,
    'N_ur': 2.0328,
    'N_ncdm': 1,
    'm_ncdm': 0.06,
    'tau_reio': 0.0568,
    'omega_b': 0.022447,
    'omega_cdm': 0.11923}
#
# Create an instance of the CLASS wrapper, set the parameters and
# run the code. It should take no more than a few seconds...
cosmo = Class()
cosmo.set(params)
cosmo.compute()

In [ ]:
# And it has lots of other things it can tell you...
print("sig8={:.3f}".format(cosmo.sigma8()))
#
cosmo.get_current_derived_parameters(['H0','Omega_m','Omega_Lambda','age'])

In [ ]:
# We can also do things like the growth factor and growth rate.
for zz in np.arange(0.0,2.51,0.5):
    print("At z={:4.1f}, D(z)={:8.4f}, f(z)={:8.4f}".\
          format(zz,cosmo.scale_independent_growth_factor(zz),\
                 cosmo.scale_independent_growth_factor_f(zz)))

In [ ]:
# Going back to our fiducial model we can also look at the matter power spectrum:
zz = 1.0
kk = np.logspace(-3.0,1.0,100)
pk = np.array( [cosmo.pk(k,zz) for k in kk] )